<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/GPT-J-6B/Inference_with_GPT_J_6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GPT-J-6B推理

在本笔记本中，我们将使用EleutherAI的[GPT-J-6B模型]进行推理（即生成新文本）(https://github.com/kingoflolz/mesh-transformer-jax/)，这是一个在[The Pile]上训练的60亿参数GPT模型(https://arxiv.org/abs/2101.00027)，一个巨大的公开文本数据集，也由EleutherAI收集。该模型本身是使用JAX和Haiku（后者是JAX之上的神经网络库）在TPUv3s上训练的。

[EleutherAI](https://www.eleuther.ai/)它本身就是一个由人工智能研究人员组成的团队，他们正在进行令人惊叹的人工智能研究（并将所有内容公开并免费使用）。他们还创建了[GPT Neo](https://github.com/EleutherAI/gpt-neo)，它们是较小的GPT变体（分别具有1.25亿、13亿和27亿个参数）。在轮毂上查看他们的型号[此处](https://huggingface.co/EleutherAI).

注意：此笔记本电脑需要至少12.1GB的CPU内存。我个人使用Colab Pro来运行它（并将运行时设置为GPU——高RAM使用率）。不幸的是，Colab的免费版本只提供10GB的RAM，这是不够的。

## 负载模型和分词器
 
首先，我们从中心加载模型。我们选择“float16”版本，这意味着所有参数都使用16位存储，而不是默认的float32位（需要两倍的RAM内存）。我们还将“low_cpu_mem_use”设置为“True”（这是在[本PR]中介绍的）(https://github.com/huggingface/transformers/pull/13466))，以便只将模型加载一次到CPU内存中。

接下来，我们将模型移动到GPU并加载相应的标记器，我们将使用它为模型准备文本。

In [1]:
import os
import mindnlp
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
if "RANK_TABLE_FILE" in os.environ:
    del os.environ["RANK_TABLE_FILE"]

[WARNING] GE_ADPT(60874,ffffb07090b0,python):2024-10-14-20:29:14.312.074 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleGetModelId failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleGetModelId
[WARNING] GE_ADPT(60874,ffffb07090b0,python):2024-10-14-20:29:14.312.122 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleLoadFromMem failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleLoadFromMem
[WARNING] GE_ADPT(60874,ffffb07090b0,python):2024-10-14-20:29:14.312.139 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleUnload failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleUnload
[WARNING] GE_ADPT(60874,ffffb07090b0,python):2024-10-14-20:29:14.312.325 [mindspore/ccsrc/utils/dlopen_macro.h:163] Dl

In [2]:

from mindnlp.transformers import GPTJForCausalLM, AutoTokenizer




In [3]:
model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", low_cpu_mem_usage=True, from_pt=True)


tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

[WARNING] DEVICE(60874,ffffb07090b0,python):2024-10-14-20:32:15.834.924 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:188] CheckVmmDriverVersion] Driver version is less than 24.0.0, vmm is disabled by default, drvier_version: 23.0.6
[WARNING] DEVICE(60874,fffe0d4f71e0,python):2024-10-14-20:32:38.987.715 [mindspore/ccsrc/transform/acl_ir/op_api_convert.h:114] GetOpApiFunc] Dlsym aclSetAclOpExecutorRepeatable failed!
[WARNING] KERNEL(60874,fffe0d4f71e0,python):2024-10-14-20:32:38.987.790 [mindspore/ccsrc/transform/acl_ir/op_api_cache.h:54] SetExecutorRepeatable] The aclSetAclOpExecutorRepeatable is unavailable, which results in aclnn cache miss.
[WARNING] DEVICE(60874,fffe0ac751e0,python):2024-10-14-20:32:38.991.335 [mindspore/ccsrc/transform/acl_ir/op_api_convert.h:114] GetOpApiFunc] Dlsym aclDestroyAclOpExecutor failed!
Some weights of the model checkpoint at EleutherAI/gpt-j-6B were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'tran

## 推理

在这里，我们可以提供一个自定义提示，使用模型的标记器准备该提示（模型所需的唯一输入是`input_ids`）。然后，我们将`input_ids`也移动到GPU，并使用`.generate（）`方法自回归生成令牌。请注意，此方法支持各种解码方法，包括波束搜索和前k采样。所有细节都可以在这篇[博客文章]中找到(https://huggingface.co/blog/how-to-generate).

In [4]:
prompt = "The Belgian national football team "
input_ids = tokenizer(prompt, return_tensors="ms").input_ids

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


The Belgian national football team  () represents  in international football competition, and they are governed by the Royal Belgian Football Association. Belgian football began in the 1930s and was professionalized in 1992. Most of Belgium's best players have been born abroad, due to the country's geographic size; as a result, only Belgium has achieved success in major international tournaments, having never won the FIFA World Cup or any other continental tournament.

There are 16 clubs in the Belgian First Division, the highest level of professional football in the country., who are the reigning champions of Belgium, are playing at this level for the first time since 1997. The Belgian national team has never won a major international title.

History

Antwerp-born player Herrengier Deschamps was the first Belgian to play in a major tournament, the 1930 FIFA World Cup. Belgium was admitted to UEFA in 1991, and the country's first match with a non-European nation was against Japan on 7


请注意，大型GPT模型最有趣的特性之一是它们能够进行所谓的“少量学习”。这意味着，在文本提示中只给出几个示例，该模型能够快速推广到新的、看不见的示例。因此，您可以使用此模型进行少量镜头文本分类，如下所示：

In [5]:
prompt = """
Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment: """
input_ids = tokenizer(prompt, return_tensors="ms").input_ids

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment: ___________

#####

Sentence: This movie was a fun flop.
Sentiment: negative

#####

Sentence: I loved this movie.
Sentiment: positive

#####

Sentence: I hated this movie.
Sentiment: negative

#####

Sentence: I liked this movie.
Sentiment: positive

#####

Sentence: I hated this movie.
Sentiment: negative

#####

Sentence: I liked this movie.
Sentiment: positive




另一个注意事项：由于GPT-J-6B是在The Pile（其中包含大量Github代码）上训练的，因此该模型能够执行代码生成（类似于OpenAI的Codex模型）。这里有一个例子：

In [6]:
prompt = """Instruction: Generate a Python function that lets you reverse a list.

Answer: """
input_ids = tokenizer(prompt, return_tensors="ms").input_ids

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruction: Generate a Python function that lets you reverse a list.

Answer: 

What we need is a list comprehension that takes the entire list as an argument. We also need to take every element in that list. You can do this with a list comprehension, like so: 
In [17]: def reverse_list(l): 
             return [x[::-1] for x in l] 
            . 

Then we can call our list comp:
In [18]: reverse_1_1(range(10)) 
Out[18]: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0] 

In [19]: reverse_1_2(range(10)) 
Out[19]: [0, 1, 2, 3


## 批量推理

您还可以对批量提示执行推理，如下所示：

In [7]:
# this is a single input batch with size 3
texts = ["Once there was a man ", "The weather today will be ", "The best football player in the world is "]

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
encoding = tokenizer(texts, padding=True, return_tensors='ms')

generated_ids = model.generate(**encoding, do_sample=True, temperature=0.9, max_length=50)
generated_texts = tokenizer.batch_decode(
    generated_ids, skip_special_tokens=True)

for text in generated_texts:
  print("---------")
  print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------
Once there was a man  
Who could not find his way home

Now the man who could not find his way  
Home because he was lost  
Does not have to wonder  
Where the
---------
The weather today will be 
- it's a 30/50% chance of rain.
- The current temperature is 18°C.
- The wind speed will be around 12 km/h.
- The current humidity
---------
The best football player in the world is _______.

1-David BeckhamThe 2nd option is a no brainer to me. The best football player in the world is of course David Beckham. David has played for a variety of teams
